In [1]:

import os
import platform
import dask
import distributed
import time
from dask.distributed import Client, LocalCluster, as_completed
from random import *
class ESWorker:
    """ A simple class representing an athena event worker """
    n = 0
    myid = 0
    buffer = list(range(100000)) #add some baggage to the object

    def __init__(self, id, buffer):
        self.n = 0
        self.myid = id
        self.buffer = buffer
        print("Worker constructor for ", self, " id ", self.myid, flush=True)
    def id(self):
        return self.myid
    
    def processed(self):
        return self.n

    def increment(self):
        self.n += 1
        return self.n
    
    def processEvent(self, event):
        soFar = self.increment()
        #do something
#         for i in range(10000):
#             self.buffer.pop()
#         if ((event == 17 or event == 13) and random() < 0.5):
#             print(event, " is not a lucky number")
#             raise EventException(event,self) 
        pid = os.getpid()
        hostname = platform.node()    
        print(pid, hostname,time.time(), "Worker ", self, self.id(), " done with event ", event, "! ",
              self.processed(), " events done so far", flush=True)
        return(self, event)
        
class EventException(Exception):
       def __init__(self, event, worker):
            self.event = event
            self.worker = worker
            
        
def processEvent(event,worker):
    pid = os.getpid()
    hostname = platform.node()
    print(pid, hostname, time.time(), " calling processEvent on Worker ", worker.id(), flush=True)
    #(w, e) = (worker.processEvent(event)).result()
    (w, e) = (worker.processEvent(event))
    print(pid, hostname, time.time(), " done with processEvent on Worker ", w.id(), flush=True)
    return (w, e) 



In [2]:
# n_slots=3
# n_workers=n_slots
# cluster = LocalCluster('10.128.1.22:34423',n_workers + n_slots)
# print(cluster)
# client = Client(cluster)

# print(client)


6(6, 'tcp://127.0.0.1:36219', workers=4, threads=8, memory=7.63 GiB)
<Client: 'tcp://127.0.0.1:36219' processes=4 threads=8, memory=7.63 GiB>


In [ ]:
#run this block if you are running this program in Cori after these inputs:
'''
cd $SCRATCH
module load nersc-dask
start-dask-mpi --ntasks=256 --image=stephey/nersc-dask-example:v0.6.0
'''

scheduler_file = os.path. join(os.environ["SCRATCH"], "scheduler.json")
dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status"

n_slots = 3
client = Client(scheduler_file="scheduler.json") 
print(client)

In [3]:
events=[e for e in range(20)]
print(events)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [4]:
#workers = client.submit(ESWorker, 88)
#workers = [client.submit(ESWorker, w, actor=True) for w in range(n_workers)]
#workers = client.map(ESWorker, range(n_workers), key="ESworker", actor=True)
something=list(range(10000000))
workers = [ESWorker(w, something) for w in range(n_slots)]
print(workers)


Worker constructor for  <__main__.ESWorker object at 0x7f4770063be0>  id  0
Worker constructor for  <__main__.ESWorker object at 0x7f4770063d30>  id  1
Worker constructor for  <__main__.ESWorker object at 0x7f47700633d0>  id  2
[<__main__.ESWorker object at 0x7f4770063be0>, <__main__.ESWorker object at 0x7f4770063d30>, <__main__.ESWorker object at 0x7f47700633d0>]


In [5]:
#slots = [client.submit(processEvent, event=events.pop(), worker=workers.pop()) for s in range(n_slots)]
slots = client.map(processEvent, [events.pop() for s in range(n_slots)] , workers, key="slot")
# print(slots)
# print(events)


/home/calaf/.local/lib/python3.8/site-packages/distributed/worker.py:3801: UserWarning: Large object of size 47.94 MiB detected in task graph: 
  (19, <__main__.ESWorker object at 0x7f4770063be0>)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


[<Future: pending, key: slot-7596ef3e5e7cd1353ae1bb1938719be1>, <Future: pending, key: slot-58c483360c42919c5d2e39152a387c0c>, <Future: pending, key: slot-1c2e3dfc3f856a6467fe1562a2746a2a>]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [6]:
seq = as_completed(slots)
print(seq)

   

<as_completed: waiting=3 done=0>


In [7]:
pid = os.getpid()
hostname = platform.node()
for slot in seq:
    try:
        (workerDone, evtDone) = slot.result()
        print(pid, hostname, time.time(), workerDone.id(), " returned event ", evtDone, flush=True)
    except EventException as ee:
        workerDone=ee.worker
        print("mmmhhh worker ", workerDone.id()," lost event ", ee.event) #how would we try it again?
    
    print(seq)

    #if more events left add actors back into the loop 
    if events:
        # add worker that just returned back into the loop
        seq.add(client.submit(processEvent, events.pop(), workerDone))


3292 DESKTOP-E5D74QR 1627406191.7991512  calling processEvent on Worker  1
3292 DESKTOP-E5D74QR 1627406191.8016517 Worker  <__main__.ESWorker object at 0x7f111c2a5ac0> 1  done with event  18 !  1  events done so far
3292 DESKTOP-E5D74QR 1627406191.8017023  done with processEvent on Worker  1
3297 DESKTOP-E5D74QR 1627406191.8135426  calling processEvent on Worker  0
3297 DESKTOP-E5D74QR 1627406191.8148468 Worker  <__main__.ESWorker object at 0x7fb5ba29edc0> 0  done with event  19 !  1  events done so far
3297 DESKTOP-E5D74QR 1627406191.814909  done with processEvent on Worker  0
3294 DESKTOP-E5D74QR 1627406191.883126  calling processEvent on Worker  2
3294 DESKTOP-E5D74QR 1627406191.8853793 Worker  <__main__.ESWorker object at 0x7f9844d461f0> 2  done with event  17 !  1  events done so far
3294 DESKTOP-E5D74QR 1627406191.8854632  done with processEvent on Worker  2
3273 DESKTOP-E5D74QR 1627406192.6289296 1  returned event  18
<as_completed: waiting=0 done=2>
3300 DESKTOP-E5D74QR 1627406

In [8]:
print(workerDone, workerDone.id(), workerDone.processed())
print(workers)
#wrap up        
if not events:
    print("DONE: event list is empty. Events processed by each worker ", list(map(ESWorker.processed, workers)))
else:
    print("ERROR: event list not empty! ", events)

<__main__.ESWorker object at 0x7f47186e72e0> 0 7
[<__main__.ESWorker object at 0x7f4770063be0>, <__main__.ESWorker object at 0x7f4770063d30>, <__main__.ESWorker object at 0x7f47700633d0>]
DONE: event list is empty. Events processed by each worker  [0, 0, 0]
